In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import albumentations as A
from matplotlib import pyplot as plt
%matplotlib inline

In [10]:
from tensorflow import keras
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [3]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose
)

In [4]:
tf.__version__

'1.14.0'

### Some preprocessing helping func's

In [5]:
def augment(aug,image):
    image = aug(image=image)['image']
    return image
    
def augment_and_show(aug,image):
    image = augment(aug,image)
    plt.figure(figsize=(10,10))
    plt.imshow(image)

In [6]:
def augment_2d(inputs,p=.5):
    return augment(Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p),inputs)
    

In [7]:
def find_in_dir(dirname):
    return [os.path.join(dirname, fname) for fname in sorted(os.listdir(dirname))]


In [8]:
train_datagen = ImageDataGenerator()

train_generator = ImageDataGenerator().flow_from_directory(
'images',
target_size = (256,256),
batch_size = 64)

Found 194920 images belonging to 39 classes.


In [11]:
model = Sequential()
model.add(Lambda(augment_2d,input_shape
                =train_generator.image_shape))
model.add(Conv2D(3,6,5))
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(6,16,5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(120))
model.add(Activation('relu'))
model.add(Dense(84))
model.add(Activation('relu'))
model.add(Dense(39))
model.add(Activation('softmax'))

W0825 19:51:26.655797  9544 deprecation.py:506] From C:\conda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
opt = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])
model.fit_generator(train_generator,epochs=10)

Epoch 1/10
  61/3046 [..............................] - ETA: 6:08:19 - loss: 5.3144 - acc: 0.0264